In [12]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
pwd

'/tf/image-classification/src/trainer'

In [3]:
base_dir = '../../../dataset/dist/'

train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

train_accepted_dir = os.path.join(train_dir, 'accepted')

train_rejected_dir = os.path.join(train_dir, 'rejected')

valid_accepted_dir = os.path.join(valid_dir, 'accepted')

valid_rejected_dir = os.path.join(valid_dir, 'rejected')

In [4]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [5]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory( valid_dir,  batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 4361 images belonging to 2 classes.
Found 546 images belonging to 2 classes.


In [6]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [7]:
for layer in base_model.layers:
    layer.trainable = False

In [8]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(1, activation='sigmoid'))

In [9]:
base_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
resnet_history = base_model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 100)

Epoch 1/100
100/100 [==============================] - 26s 168ms/step - loss: 2.0070 - accuracy: 0.5396 - val_loss: 1.0153 - val_accuracy: 0.5000
Epoch 2/100
100/100 [==============================] - 16s 158ms/step - loss: 1.5750 - accuracy: 0.6070 - val_loss: 2.6712 - val_accuracy: 0.5000
Epoch 3/100
100/100 [==============================] - 16s 157ms/step - loss: 1.4086 - accuracy: 0.6436 - val_loss: 3.4311 - val_accuracy: 0.5000
Epoch 4/100
100/100 [==============================] - 16s 155ms/step - loss: 1.3274 - accuracy: 0.6655 - val_loss: 3.0241 - val_accuracy: 0.5018
Epoch 5/100
100/100 [==============================] - 16s 155ms/step - loss: 1.2236 - accuracy: 0.6759 - val_loss: 3.4134 - val_accuracy: 0.5201
Epoch 6/100
100/100 [==============================] - 16s 160ms/step - loss: 1.1466 - accuracy: 0.6845 - val_loss: 2.3886 - val_accuracy: 0.5495
Epoch 7/100
100/100 [==============================] - 15s 154ms/step - loss: 1.1205 - accuracy: 0.7017 - val_loss: 1.3550 -

Epoch 57/100
100/100 [==============================] - 16s 155ms/step - loss: 0.4817 - accuracy: 0.8225 - val_loss: 0.6002 - val_accuracy: 0.8040
Epoch 58/100
100/100 [==============================] - 16s 155ms/step - loss: 0.5044 - accuracy: 0.8160 - val_loss: 0.5143 - val_accuracy: 0.8150
Epoch 59/100
100/100 [==============================] - 16s 155ms/step - loss: 0.5032 - accuracy: 0.8245 - val_loss: 0.5127 - val_accuracy: 0.8187
Epoch 60/100
100/100 [==============================] - 15s 153ms/step - loss: 0.4530 - accuracy: 0.8299 - val_loss: 0.5264 - val_accuracy: 0.8150
Epoch 61/100
100/100 [==============================] - 16s 155ms/step - loss: 0.5373 - accuracy: 0.8105 - val_loss: 0.5093 - val_accuracy: 0.8278
Epoch 62/100
100/100 [==============================] - 16s 156ms/step - loss: 0.4869 - accuracy: 0.8295 - val_loss: 0.5794 - val_accuracy: 0.8077
Epoch 63/100
100/100 [==============================] - 15s 155ms/step - loss: 0.4966 - accuracy: 0.8274 - val_loss: 0

In [11]:
base_model.save('100epoch.h5')